In [1]:
import sys
sys.path.append('../')

from envs.GridMaps import *
from utilities.transitionDatasetGeneration import *

In [2]:
shape = "14x14"
NoWalls = True
if shape == "5x5":
    maze = maze5x5["maze"]
    n_epochs = 15000
    folder_name = "5x5_5_model"
    n_models = 5
    hidden_size = 32

elif shape == "14x14":
    if NoWalls:
        maze = NoWalls_maze14x14["maze"]
        n_epochs = 120000
        folder_name = "NoWalls_5_model"  #to store the models weights and losses
    else:
        maze = maze14x14["maze"]
        n_epochs = 120000
        folder_name = "5_model"
    n_models = 5
    hidden_size = 512
    

train_transitions, val_transitions = transition_generator(maze)

In [3]:
models_arr = []

for i in range(n_models):
    print(f"Training model {i+1}...")

    model = ModelTrainer(train_transitions, val_transitions, n_epochs, hidden_size, rand_seed=True)

    model.fit()
    models_arr.append(model)

Training model 1...


100%|██████████| 120000/120000 [53:23<00:00, 37.45it/s]


Trained successfully!
Final loss: 0.00031779237906448543

Training model 2...


100%|██████████| 120000/120000 [24:16<00:00, 82.37it/s]


Trained successfully!
Final loss: 9.627534382161684e-06

Training model 3...


...

Trained successfully!
Final loss: 9.525135101284832e-05

Training model 4...


100%|██████████| 120000/120000 [24:09<00:00, 82.80it/s]


Trained successfully!
Final loss: 7.87990793469362e-05

Training model 5...


100%|██████████| 120000/120000 [24:03<00:00, 83.11it/s]

Trained successfully!
Final loss: 4.033895947941346e-06



In [4]:
store_models(models_arr, folder_name)

In [5]:
print('\nEVALUATION:\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

arr1 = []
arr1_raw = []
arr2 = []
arr2_raw = []

for model in models_arr:
    # Single predictions
    with torch.no_grad():
        # Move the input data to the GPU
        inp1 = torch.FloatTensor([[5, 3, 3]]).to(device)
        inp2 = torch.FloatTensor([[5, 4, 3]]).to(device)
        
        # Perform inference on the GPU
        resultado1 = model.model(inp1)
        resultado2 = model.model(inp2)
        
        # Move the result back to CPU and convert to numpy
        resultado1 = resultado1.cpu().detach().numpy()
        resultado2 = resultado2.cpu().detach().numpy()

        arr1.append(np.int32(np.round(resultado1[0])))
        arr1_raw.append(resultado1[0])
        arr2.append(np.int32(np.round(resultado2[0])))
        arr2_raw.append(resultado2[0])
        
for elem, elem2 in zip(arr1, arr2):
    print(elem, '         ', elem2)

#whats the mean and the std of the raw values?
print('\n')
print('Mean of raw values:')
print(np.mean(arr1_raw, axis=0))
print(np.mean(arr2_raw, axis=0))
print('\n')
print('Std of raw values:')
print(np.std(arr1_raw, axis=0), '     ', np.linalg.norm(np.std(arr1_raw, axis=0)))
print(np.std(arr2_raw, axis=0), '     ', np.linalg.norm(np.std(arr2_raw, axis=0)))



EVALUATION:

Using device: cuda
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]


Mean of raw values:
[4.978347 4.020192]
[4.999214  4.9975233]


Std of raw values:
[0.02268483 0.02357475]       0.032716516
[0.02118679 0.01715924]       0.02726389
